In [2]:
from datasets import load_dataset

# Login using e.g. `huggingface-cli login` to access this dataset
ds = load_dataset("agents-course/unit3-invitees")

README.md:   0%|          | 0.00/371 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/3.32k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/3 [00:00<?, ? examples/s]

# Load and prepare dataset

# Create the retriever tool

# Integrate the Tool


In [4]:
import datasets
from langchain.docstore.document import Document

# Load the dataset
guest_dataset = datasets.load_dataset("agents-course/unit3-invitees", split="train")

# Convert dataset entries into Document objects
docs = [
    Document(
        page_content="\n".join([
            f"Name: {guest['name']}",
            f"Relation: {guest['relation']}",
            f"Description: {guest['description']}",
            f"Email: {guest['email']}"
        ]),
        metadata={"name": guest["name"]}
    )
    for guest in guest_dataset
]

In [5]:
#uses the BM25Retriever
from langchain_community.retrievers import BM25Retriever
from langchain.tools import Tool
bm25_retriever=BM25Retriever.from_documents(docs)
def extract_text(query: str)->str:
    """Retrieves detailed information about gala guests based on their name or relation."""
    results=bm25_retriever.invoke(query)
    if results:
        return "\n\n".join([doc.page_content for doc in results[:3]])
    else:
        return "No matching guest information found."
guest_info_tool=Tool(
    name="guest_info_retriever",
    func=extract_text,
    description="Retrieves detailed information about gala guests based on their name or relation."
)

In [14]:
# Adding weather tool
import random
from smolagents import Tool
class weatherInfoTool(Tool):
    name='Weather Info'
    description='Tool to fetch the weather information about the given place.'
    inputs={
        "location":{
            'type':'string',
            'info':'The place for which the weather is to be searched for.'
        }
    }
    output_type='string'
    def forward(self,location):
        weather_conditions=[
            {"condition":"Sunny","temp_c":25.6},
            {"condition":"Rainy","temp_c":19.4},
            {"condition":"Windy","temp_c":22}
            ]
        data=random.choice(weather_conditions)
        return f"Weather in {location} is {data['condtions']} and the temperature is {data['temp_c']} celsius "
# adding hub_stat tool
from huggingface_hub import list_models
class HubStatTool(Tool):
    name='Hub Stat Tool'
    description="Tool used to fetch the most downloaded model and it's stats from HuggingFace"
    inputs={
        "author":{
            "type":"string",
            "description":"The creator of the model. The username of the author or organization that provides the model."}}
    output_type="string"
    def forward(self,author):
        try:
            models=list(list_models(author=author,sort='downloads',direction=-1,limit=1))
            if models:
                model=models[0]
                return f"The most downloaded model by {author} is {model.id} and has {model.downloads} downloads."
            else:
                return f"No models found for {author}"
        except Exception as e:
            return f"Error fetching model stats for {author}: {str(e)}"

In [16]:
import os
from typing import TypedDict, Annotated
from langgraph.graph.message import add_messages
from langchain_core.messages import AnyMessage,HumanMessage,AIMessage
from langgraph.prebuilt import ToolNode
from langgraph.graph import START, StateGraph
from langgraph.prebuilt import tools_condition
from langchain_huggingface import HuggingFaceEndpoint, ChatHuggingFace
from smolagents import DuckDuckGoSearchTool
DuckDuckGoSearchRun=DuckDuckGoSearchTool()
llm= HuggingFaceEndpoint(
    repo_id="Qwen/Qwen2.5-Coder-32B-Instruct",
    huggingfacehub_api_token=os.environ.get("HUGGING_FACE_API_KEY")
)
chat=ChatHuggingFace(llm=llm,verbose=True)
tools=[guest_info_tool,weatherInfoTool,HubStatTool]
chat_with_tools=chat.bind_tools(tools)
#Generating the AgentState and Agent Graph
class AgentState(TypedDict):
    messages: Annotated[list[AnyMessage],add_messages]
def assistant(state: AgentState):
    return{
        "messages": [chat_with_tools.invoke(state["messages"])],
    }
#Graph
builder=StateGraph(AgentState)
#Define nodes: thses do the work
builder.add_node("assistant",assistant)
builder.add_node("tools",ToolNode(tools))
#Define edges: these determine how the control flow moves
builder.add_edge(START,"assistant")
builder.add_conditional_edges(
    "assistant",
    #if tool required; route to tools
    # Otherwise provide a direct response
    tools_condition,
)
builder.add_edge("tools",'assistant')
alfred=builder.compile()
messages=[HumanMessage(content="Tell me about our guest named ' Lady Ada Lovelace'.")]
response=alfred.invoke({"messages":messages})
print("Alfred's response:")
print(response['messages'][-1].content)

HfHubHTTPError: 402 Client Error: Payment Required for url: https://router.huggingface.co/nebius/v1/chat/completions (Request ID: Root=1-685e4829-47474601011a18a4502435fc;e43d1da9-16ce-4597-81d6-d590edef348a)

You have exceeded your monthly included credits for Inference Providers. Subscribe to PRO to get 20x more monthly included credits.

In [17]:
# First interaction
response = alfred.invoke({"messages": [HumanMessage(content="Tell me about 'Lady Ada Lovelace'. What's her background and how is she related to me?")]})


print("🎩 Alfred's Response:")
print(response['messages'][-1].content)
print()

# Second interaction (referencing the first)
response = alfred.invoke({"messages": response["messages"] + [HumanMessage(content="What projects is she currently working on?")]})

print("🎩 Alfred's Response:")
print(response['messages'][-1].content)

HfHubHTTPError: 402 Client Error: Payment Required for url: https://router.huggingface.co/nebius/v1/chat/completions (Request ID: Root=1-685e48cb-244dfd02792c4225325c75a2;8749bdc6-33d7-4b51-87fb-19ed2fbbb211)

You have exceeded your monthly included credits for Inference Providers. Subscribe to PRO to get 20x more monthly included credits.